### Dataframe operations

In [1]:
from pyspark.sql import Row

row = Row(name="Alice", age=11)
print row
print row['name'], row['age']
print row.name, row.age

row = Row(name="Alice", age=11, count=1)
print row.count
print row['count']

Row(age=11, name='Alice')
Alice 11
Alice 11
<built-in method count of Row object at 0x10cd59470>
1


In [2]:
# Data file at https://www.cse.ust.hk/msbd/data/building.csv
df = spark.read.csv('../data/building.csv', header=True, inferSchema=True)

In [3]:
# show the content of the dataframe
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [4]:
# Print the dataframe schema in a tree format
df.printSchema()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
# Create an RDD from the dataframe
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr=u'M1', BuildingAge=25, HVACproduct=u'AC1000', Country=u'USA'), Row(BuildingID=2, BuildingMgr=u'M2', BuildingAge=27, HVACproduct=u'FN39TG', Country=u'France'), Row(BuildingID=3, BuildingMgr=u'M3', BuildingAge=28, HVACproduct=u'JDNS77', Country=u'Brazil')]

In [7]:
# Retrieve specific columns from the dataframe
df.select('BuildingID', 'Country').show(3)

+----------+-------+
|BuildingID|Country|
+----------+-------+
|         1|    USA|
|         2| France|
|         3| Brazil|
+----------+-------+
only showing top 3 rows



In [9]:
from pyspark.sql.functions import *

df.where("Country='USA'").select('*', lit('OK')).show()

+----------+-----------+-----------+-----------+-------+---+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|Country| OK|
+----------+-----------+-----------+-----------+-------+---+
|         1|         M1|         25|     AC1000|    USA| OK|
+----------+-----------+-----------+-----------+-------+---+



In [1]:
# Use GroupBy clause with dataframe 
df.groupBy('HVACProduct').count().show()

+-----------+-----+
|HVACProduct|count|
+-----------+-----+
|    ACMAX22|    4|
|     AC1000|    4|
|     JDNS77|    4|
|     FN39TG|    4|
|     GG1919|    4|
+-----------+-----+



### Rewriting SQL with DataFrame API

The data files have been put to a public blob container, which can be accessed as follows

In [2]:
# Load data from csv files
# Data files at https://www.cse.ust.hk/msbd5003/data

dfCustomer = spark.read.csv('../data/Customer.csv', header=True, inferSchema=True)
dfProduct = spark.read.csv('../data/Product.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('../data/SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('../data/SalesOrderHeader.csv', header=True, inferSchema=True)

In [5]:
# SELECT ProductID, Name, ListPrice 
# FROM Product 
# WHERE Color = 'black'

dfProduct.filter("Color = 'Black'")\
         .select('ProductID', 'Name', 'ListPrice')\
         .show(3,truncate=False)

+---------+-------------------------+---------+
|ProductID|Name                     |ListPrice|
+---------+-------------------------+---------+
|680      |HL Road Frame - Black, 58|1431.5   |
|708      |Sport-100 Helmet, Black  |34.99    |
|722      |LL Road Frame - Black, 58|337.22   |
+---------+-------------------------+---------+
only showing top 3 rows



In [6]:
# dfProduct['ProductID'] is same as dfProduct.ProductID
# alias corresponds as in SQL
dfProduct.where(dfProduct.Color=='Black') \
         .select(dfProduct.ProductID, dfProduct['Name'], (dfProduct.ListPrice * 2).alias('DoublePrice')) \
         .show(3,truncate=False)

+---------+-------------------------+-----------+
|ProductID|Name                     |DoublePrice|
+---------+-------------------------+-----------+
|680      |HL Road Frame - Black, 58|2863.0     |
|708      |Sport-100 Helmet, Black  |69.98      |
|722      |LL Road Frame - Black, 58|674.44     |
+---------+-------------------------+-----------+
only showing top 3 rows



In [7]:
dfProduct.where(dfProduct.ListPrice * 2 > 100) \
         .select(dfProduct.ProductID, dfProduct['Name'], dfProduct.ListPrice * 2) \
         .show(3,truncate=False)

+---------+-------------------------+---------------+
|ProductID|Name                     |(ListPrice * 2)|
+---------+-------------------------+---------------+
|680      |HL Road Frame - Black, 58|2863.0         |
|706      |HL Road Frame - Red, 58  |2863.0         |
|717      |HL Road Frame - Red, 62  |2863.0         |
+---------+-------------------------+---------------+
only showing top 3 rows



In [8]:
# SELECT ProductID, Name, ListPrice 
# FROM Product 
# WHERE Color = 'black' 
# ORDER BY ProductID

dfProduct.filter("Color = 'Black'")\
         .select('ProductID', 'Name', 'ListPrice')\
         .orderBy('ListPrice')\
         .show(3,truncate=False)

+---------+---------------------+---------+
|ProductID|Name                 |ListPrice|
+---------+---------------------+---------+
|860      |Half-Finger Gloves, L|24.49    |
|858      |Half-Finger Gloves, S|24.49    |
|859      |Half-Finger Gloves, M|24.49    |
+---------+---------------------+---------+
only showing top 3 rows



In [9]:
# Find all orders and details on black product,
# return the product SalesOrderID, SalesOrderDetailID, Name, UnitPrice, and OrderQty

# SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
# FROM SalesLT.SalesOrderDetail, SalesLT.Product
# WHERE SalesOrderDetail.ProductID = Product.ProductID AND Color = 'Black'

# SELECT SalesOrderID, SalesOrderDetailID, Name, UnitPrice, OrderQty 
# FROM SalesLT.SalesOrderDetail
# JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
# WHERE Color = 'Black'

# Spark SQL supports natural joins

dfDetail.join(dfProduct, 'ProductID') \
        .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty') \
        .filter("Color='Black'")\
        .show(3)

# If we move the filter to after select, it still works.  Why?
#  lazy evaluation!

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71780|            110620|Mountain-500 Blac...|  323.994|       1|
|       71780|            110621|LL Mountain Frame...|  149.874|       1|
|       71780|            110622|HL Mountain Frame...|   809.76|       1|
+------------+------------------+--------------------+---------+--------+
only showing top 3 rows



In [10]:
# This also works:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d2 = d1.filter("Color = 'Black'")
d2.show(3)
d2.explain()

+------------+------------------+--------------------+---------+--------+
|SalesOrderID|SalesOrderDetailID|                Name|UnitPrice|OrderQty|
+------------+------------------+--------------------+---------+--------+
|       71780|            110620|Mountain-500 Blac...|  323.994|       1|
|       71780|            110621|LL Mountain Frame...|  149.874|       1|
|       71780|            110622|HL Mountain Frame...|   809.76|       1|
+------------+------------------+--------------------+---------+--------+
only showing top 3 rows

== Physical Plan ==
*Project [SalesOrderID#175, SalesOrderDetailID#176, Name#129, UnitPrice#179, OrderQty#177]
+- *BroadcastHashJoin [ProductID#178], [ProductID#128], Inner, BuildRight
   :- *Project [SalesOrderID#175, SalesOrderDetailID#176, OrderQty#177, ProductID#178, UnitPrice#179]
   :  +- *Filter isnotnull(ProductID#178)
   :     +- *FileScan csv [SalesOrderID#175,SalesOrderDetailID#176,OrderQty#177,ProductID#178,UnitPrice#179] Batched: false, For

In [14]:
# This will report an error:

d1 = dfDetail.join(dfProduct, 'ProductID') \
             .select('SalesOrderID', 'SalesOrderDetailID', 'Name', 'UnitPrice', 'OrderQty')
d1.write.csv('temp.csv', mode = 'overwrite', header = True)
d2 = spark.read.csv('temp.csv', header = True, inferSchema = True)
# d2 doesn't have color column
d2.filter("Color = 'Black'").show()


In [15]:
# Find all orders that include at least one black product, 
# return the product SalesOrderID, Name, UnitPrice, and OrderQty

# SELECT DISTINCT SalesOrderID
# FROM SalesLT.SalesOrderDetail
# JOIN SalesLT.Product ON SalesOrderDetail.ProductID = Product.ProductID
# WHERE Color = 'Black'

dfDetail.join(dfProduct.filter("Color='Black'"), 'ProductID') \
        .select('SalesOrderID') \
        .distinct() \
        .show(3)

+------------+
|SalesOrderID|
+------------+
|       71902|
|       71832|
|       71915|
+------------+
only showing top 3 rows



In [16]:
# How many colors in the products?

# SELECT COUNT(DISTINCT Color)
# FROM SalesLT.Product

dfProduct.select('Color').distinct().count()

# It's 1 more than standard SQL.  In standard SQL, COUNT() does not count NULLs.

10

In [17]:
# Find the total price of each order, 
# return SalesOrderID and total price (column name should be ‘totalprice’)

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail
# GROUP BY SalesOrderID

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail.OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .show(3)

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71867|             858.9|
|       71902|59894.209199999976|
|       71832|      28950.678108|
+------------+------------------+
only showing top 3 rows



In [18]:
# Find the total price of each order where the total price > 10000

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail
# GROUP BY SalesOrderID
# HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail. OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice > 10000')\
        .show(3)

+------------+------------------+
|SalesOrderID|        TotalPrice|
+------------+------------------+
|       71902|59894.209199999976|
|       71832|      28950.678108|
|       71858|11528.844000000001|
+------------+------------------+
only showing top 3 rows



In [19]:
# Find the total price on the black products of each order where the total price > 10000

# SELECT SalesOrderID, SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) AS TotalPrice
# FROM SalesLT.SalesOrderDetail, SalesLT.Product
# WHERE SalesLT.SalesOrderDetail.ProductID = SalesLT.Product.ProductID AND Color = 'Black'
# GROUP BY SalesOrderID
# HAVING SUM(UnitPrice*OrderQty*(1-UnitPriceDiscount)) > 10000

dfDetail.select('*', (dfDetail.UnitPrice * dfDetail. OrderQty
                      * (1 - dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .join(dfProduct.where("Color = 'Black'"), 'ProductID') \
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        .where('TotalPrice > 10000')\
        .show(3)

+------------+------------+
|SalesOrderID|  TotalPrice|
+------------+------------+
|       71902|   26677.884|
|       71832|16883.748108|
|       71938|   33779.448|
+------------+------------+
only showing top 3 rows



In [5]:
# For each customer, find the total quantity of black products bought.
# Report CustomerID, FirstName, LastName, and total quantity

# select saleslt.customer.customerid, FirstName, LastName, sum(orderqty)
# from saleslt.customer
# left outer join 
# (
# saleslt.salesorderheader
# join saleslt.salesorderdetail
# on saleslt.salesorderdetail.salesorderid = saleslt.salesorderheader.salesorderid
# join saleslt.product
# on saleslt.product.productid = saleslt.salesorderdetail.productid and color = 'black'
# )
# on saleslt.customer.customerid = saleslt.salesorderheader.customerid
# group by saleslt.customer.customerid, FirstName, LastName
# order by sum(orderqty) desc

d1 = dfDetail.join(dfProduct, 'ProductID')\
             .where('Color = "Black"') \
             .join(dfHeader, 'SalesOrderID')\
             .groupBy('CustomerID').sum('OrderQty')
dfCustomer.join(d1, 'CustomerID', 'left_outer')\
          .select('CustomerID', 'FirstName', 'LastName', 'sum(OrderQty)')\
          .orderBy('sum(OrderQty)', ascending=False)\
          .show(3)

+----------+---------+------------+-------------+
|CustomerID|FirstName|    LastName|sum(OrderQty)|
+----------+---------+------------+-------------+
|     30050|  Krishna|Sunkammurali|           89|
|     29796|      Jon|      Grande|           65|
|     29957|    Kevin|         Liu|           62|
+----------+---------+------------+-------------+
only showing top 3 rows



-------

### Embed SQL queries

You can also run SQL queries over dataframes once you register them as temporary tables within the SparkSession.

In [10]:
# Register the dataframe as a temporary view called HVAC
df.createOrReplaceTempView('HVAC')

In [11]:
spark.sql('SELECT * FROM HVAC WHERE BuildingAge >= 10').show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|        14|        M14|         17|     GG1919|     Germany|
|        15|        M15|         19|    ACMAX22|      Israel|
|       

In [12]:
# Can even mix DataFrame API with SQL:
df.where('BuildingAge >= 10').createOrReplaceTempView('OldBuildings')
spark.sql('SELECT HVACproduct, COUNT(*) FROM OldBuildings GROUP BY HVACproduct').show()

+-----------+--------+
|HVACproduct|count(1)|
+-----------+--------+
|    ACMAX22|       3|
|     AC1000|       3|
|     JDNS77|       4|
|     FN39TG|       4|
|     GG1919|       4|
+-----------+--------+



In [13]:
d1 = spark.sql('SELECT * FROM HVAC WHERE BuildingAge >= 10')
d1.groupBy('HVACproduct').count().show()

+-----------+-----+
|HVACproduct|count|
+-----------+-----+
|    ACMAX22|    3|
|     AC1000|    3|
|     JDNS77|    4|
|     FN39TG|    4|
|     GG1919|    4|
+-----------+-----+



In [24]:
# UDF

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

slen = udf(lambda s: len(s)+2, IntegerType())
df.select('*', slen(df['Country']).alias('slen')).show()

+----------+-----------+-----------+-----------+------------+----+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|slen|
+----------+-----------+-----------+-----------+------------+----+
|         1|         M1|         25|     AC1000|         USA|   5|
|         2|         M2|         27|     FN39TG|      France|   8|
|         3|         M3|         28|     JDNS77|      Brazil|   8|
|         4|         M4|         17|     GG1919|     Finland|   9|
|         5|         M5|          3|    ACMAX22|   Hong Kong|  11|
|         6|         M6|          9|     AC1000|   Singapore|  11|
|         7|         M7|         13|     FN39TG|South Africa|  14|
|         8|         M8|         25|     JDNS77|   Australia|  11|
|         9|         M9|         11|     GG1919|      Mexico|   8|
|        10|        M10|         23|    ACMAX22|       China|   7|
|        11|        M11|         14|     AC1000|     Belgium|   9|
|        12|        M12|         26|     FN39TG|     Finland| 

In [15]:
spark.udf.register('slen', lambda s: len(s), IntegerType())
spark.sql('SELECT *, slen(Country) AS slen FROM HVAC').show()

+----------+-----------+-----------+-----------+------------+----+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|slen|
+----------+-----------+-----------+-----------+------------+----+
|         1|         M1|         25|     AC1000|         USA|   3|
|         2|         M2|         27|     FN39TG|      France|   6|
|         3|         M3|         28|     JDNS77|      Brazil|   6|
|         4|         M4|         17|     GG1919|     Finland|   7|
|         5|         M5|          3|    ACMAX22|   Hong Kong|   9|
|         6|         M6|          9|     AC1000|   Singapore|   9|
|         7|         M7|         13|     FN39TG|South Africa|  12|
|         8|         M8|         25|     JDNS77|   Australia|   9|
|         9|         M9|         11|     GG1919|      Mexico|   6|
|        10|        M10|         23|    ACMAX22|       China|   5|
|        11|        M11|         14|     AC1000|     Belgium|   7|
|        12|        M12|         26|     FN39TG|     Finland| 

---
## Flexible Data Model

Sample data file at

https://www.cse.ust.hk/msbd5003/data/products.json

In [16]:
df = spark.read.json('../data/products.json')
df.printSchema()

root
 |-- dimensions: struct (nullable = true)
 |    |-- height: double (nullable = true)
 |    |-- length: double (nullable = true)
 |    |-- width: double (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- warehouseLocation: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)



In [17]:
df.show()

+--------------+---+----------------+-----+-----------+-----------------+
|    dimensions| id|            name|price|       tags|warehouseLocation|
+--------------+---+----------------+-----+-----------+-----------------+
|[9.5,7.0,12.0]|  2|An ice sculpture| 12.5|[cold, ice]|    [-78.75,20.4]|
| [1.0,3.1,1.0]|  3|    A blue mouse| 25.5|       null|     [54.4,-32.7]|
+--------------+---+----------------+-----+-----------+-----------------+



In [18]:
# Accessing nested fields

df.select(df['dimensions.height']).show()

+------+
|height|
+------+
|   9.5|
|   1.0|
+------+



In [19]:
df.select('dimensions.height').show()

+------+
|height|
+------+
|   9.5|
|   1.0|
+------+



In [20]:
df.select('dimensions.height')\
  .filter("tags[0] = 'cold' AND warehouseLocation.latitude < 0")\
  .show()

+------+
|height|
+------+
|   9.5|
+------+



In [21]:
df.rdd.take(3)

[Row(dimensions=Row(height=9.5, length=7.0, width=12.0), id=2, name=u'An ice sculpture', price=12.5, tags=[u'cold', u'ice'], warehouseLocation=Row(latitude=-78.75, longitude=20.4)), Row(dimensions=Row(height=1.0, length=3.1, width=1.0), id=3, name=u'A blue mouse', price=25.5, tags=None, warehouseLocation=Row(latitude=54.4, longitude=-32.7))]

---
## Converting between RDD and DataFrame

Sample data file at:

https://www.cse.ust.hk/msbd5003/data/people.txt

In [22]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("../data/people.txt")

def parse(l):
    a = l.split(',')
    return (a[0], int(a[1]))

rdd = lines.map(parse)
rdd.collect()

[(u'Michael', 29), (u'Andy', 30), (u'Justin', 19)]

In [23]:
# Create the DataFrame from an RDD of tuples, schema is inferred
df = spark.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+-------+---+
|     _1| _2|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [24]:
# Create the DataFrame from an RDD of tuples with column names, type is inferred
df = spark.createDataFrame(rdd, ['name', 'age'])
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [25]:
# Create the DataFrame from an RDD of Rows, type is given in the Row objects
from pyspark.sql import Row

rdd_rows = rdd.map(lambda p: Row(name = p[0], age = p[1]))
df = spark.createDataFrame(rdd_rows)
df.printSchema()
df.show()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



In [26]:
# Row fields with types incompatible with that of previous rows will be turned into nulls
row1 = Row(name="Alice", age=11)
row2 = Row(name="Bob", age='12')
rdd_rows = sc.parallelize([row1, row2])
df1 = spark.createDataFrame(rdd_rows)
df1.show()

+----+-----+
| age| name|
+----+-----+
|  11|Alice|
|null|  Bob|
+----+-----+



In [27]:
# rdd returns the content as an RDD of Rows
teenagers = df.filter('age >= 13 and age <= 19')

teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name)
teenNames.collect()

[u'Name: Justin']

### Note:

DataFrames are stored using columnar storage with compression

RDDs are stored using row storage without compression

The RDD view of DataFrame just provides an interface, the Row objects are constructed on the fly and do not necessarily represent the internal storage format of the data

### Closure in DataFrames

In [25]:
data = range(10)
df = spark.createDataFrame(zip(data, data))
df.printSchema()
df.show()

root
 |-- _1: long (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
|  5|  5|
|  6|  6|
|  7|  7|
|  8|  8|
|  9|  9|
+---+---+



In [27]:
#??? The 'closure' behaviour in RDD doesn't seem to exist for DataFrames

x = 5
df1 = df.filter(df._1 < x)
df1.show()
x = 3
df1.show()

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
|  3|  3|
|  4|  4|
+---+---+

+---+---+
| _1| _2|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
+---+---+



In [30]:
# Because of the Catalyst optimizer !

df1.explain()

== Physical Plan ==
*Filter (isnotnull(_1#773L) && (_1#773L < 5))
+- Scan ExistingRDD[_1#773L,_2#774L]


In [34]:
def f():
    return x/2
x = 5
df1 = df.select(df._1 * 2 + f() + 1 + 1)
df1.explain()

== Physical Plan ==
*Project [((_1#773L * 2) + 4) AS ((((_1 * 2) + 2) + 1) + 1)#794L]
+- Scan ExistingRDD[_1#773L,_2#774L]


In [33]:
rdd = sc.parallelize(range(10))
x = 5
a = rdd.filter(lambda z: z < x)
print a.take(10)
x = 3
print a.take(10)

[0, 1, 2, 3, 4]
[0, 1, 2]


In [32]:
counter = 0

def increment_counter(x):
    global counter
    counter += 1

df.foreach(increment_counter)

print counter

0
